In [1]:
import csv
import numpy as np
import pandas as pd
import re

from os import path

In [17]:
rootdir = '../data/water'

input_file = 'CA-result.csv'
measure_to_measure_group = 'measure-to-measuregroup.csv'

input_base = path.splitext(input_file)[0]

bad_values = 'badvalues.csv'
unmatched_measures = 'unmatched-measures.csv'
unmapped_units = 'unmapped-units.csv'
over1k = 'over-one-thousand.csv'
less_than_zero = 'less-than-zero.csv'
result_clean = 'result-clean.csv'

def makepath(suf):
    file = input_base + '-' + suf
    print(file)
    return path.join(rootdir, file)

In [18]:
data_columns = [
    'ActivityMediaName', 
    'ActivityMediaSubdivisionName',
    'ActivityStartDate', 
    'ActivityStartTime/Time',
    'ActivityStartTime/TimeZoneCode', 
    'MonitoringLocationIdentifier', 
    'CharacteristicName',
    'ResultMeasureValue', 
    'ResultMeasure/MeasureUnitCode',
    'ResultCommentText'
]

activity_media = ['Water']

In [19]:
data = pd.read_csv(path.join(rootdir, input_file),
                  error_bad_lines=False,
                  usecols=data_columns)

len(data)

/Users/Sean/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (33,5,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


13478000

In [20]:
data["Value"] = pd.to_numeric(data.ResultMeasureValue, errors='coerce')

In [21]:
data = data[data.ActivityMediaName.isin(activity_media)]
len(data)

12392050

In [22]:
badValues = data.loc[pd.isnull(data.Value)]
badValues.to_csv(makepath(bad_values), index=False, quoting=csv.QUOTE_ALL)
length = len(badValues)
del badValues
length

CA-result-badvalues.csv


1581450

In [23]:
data = data[pd.notnull(data.Value)]
len(data)

10810600

In [24]:
measures = pd.read_csv(path.join(rootdir, measure_to_measure_group))
measures.head(1)

,Pattern,MeasureGroup,MCLG,Unit
0,antimony,Antimony,0.006,mg/L


In [25]:
%%time
for _,row in measures.iterrows():
    print(row.Pattern)
    pattern = re.compile(row.Pattern, re.IGNORECASE)
    matches = data.CharacteristicName.str.contains(row.Pattern, case=False)
    data.loc[matches, 'MeasureGroup'] = row.MeasureGroup
    data.loc[matches, 'MCLG'] = row.MCLG
    data.loc[matches, 'Unit'] = row.Unit

antimony
arsenic
barium
beryllium
cadmium
chromium
copper
fluoride
monobromoacetic acid
monochloroacetic acid
dibromoacetic acid
dichloroacetic acid
trichloroacetic acid
lead(?:-\d+)?
mercury
nitrate(?: as N)?
inorganic nitrogen.*
nitrite(?: as N)?
Aroclor.*
\d'?(?:,\d'?)*-\w*?chlorobiphenyl
total \w*?chloro biphenyls
polychlorinated biphenyls
PCB-\d+(?:\/\d+)*
\w+chlorobiphenyl
selenium
simazine
chlorodibromomethane
trichloromethane|chloroform
dichlorobromomethane
tribromomethane|bromoform
.*xylene.*
CPU times: user 6min 31s, sys: 13 s, total: 6min 44s
Wall time: 32min 29s


In [26]:
nonMatchingMeasures = data.loc[pd.isnull(data.MeasureGroup), 'CharacteristicName'].unique()
pd.DataFrame(nonMatchingMeasures).to_csv(makepath(unmatched_measures), 
                                         index=False, quoting=csv.QUOTE_ALL)
print(len(nonMatchingMeasures))
del nonMatchingMeasures

CA-result-unmatched-measures.csv
1050


In [27]:
data = data[pd.notnull(data.MeasureGroup)]
print(len(data))

742224


In [28]:
data.head(1)
data['ResultMeasure/MeasureUnitCode'].unique()

array(['ug/l', 'mg/l', 'ng/l', '% recovery', 'pg/l', 'ppb       ',
       'mg/l      ', 'ug/l      ', 'ppm       ', 'ueq/L     ', 'ueq/L',
       'ng/l      ', 'umol', 'mg/l as N', 'ug/kg', 'mg/kg', 'mg/kg as N',
       'tons/day', 'pCi/L'], dtype=object)

In [29]:
data['OriginalUnit'] = data['ResultMeasure/MeasureUnitCode'].str.strip()

In [30]:
multipliers = pd.Series({
    'mg/l': 1,
    'mg/l as N': 1,
    'mg/kg': 1,
    'mg/kg as N': 1,
    'ug/l': 1/1000,
    'ug/kg': 1/1000,
    'ng/l': 1/1000000,
    'pg/l': 1/1000000000,
    'ppm': 1,
    'ppb': 1/1000,
    'ueq/l': 62
}, name='Multiplier')
multipliers.index.name = 'OriginalUnit'

multipliers = multipliers.reset_index()

In [31]:
merged = pd.merge(data, multipliers, on='OriginalUnit', how='left')

In [32]:
unmappedUnits = merged.loc[pd.isnull(merged.Multiplier)]
unmappedUnits.to_csv(makepath(unmapped_units), index=False, quoting=csv.QUOTE_ALL)
print(len(unmappedUnits))
del unmappedUnits

CA-result-unmapped-units.csv
4045


In [33]:
merged = merged.loc[pd.notnull(merged.Multiplier)]
merged['OriginalValue'] = merged.Value
merged.Value = merged.Value * merged.Multiplier
len(merged)

738179

In [34]:
# Removing outliers - anything above 1g/L we'll assume is an outlier, 
# though it may just be bad units and we can scale down by a factor of 1,000

overOneThousand = merged.loc[merged.Value > 1000]
overOneThousand.to_csv(makepath(over1k), index=False, quoting=csv.QUOTE_ALL)
print(len(overOneThousand))
del overOneThousand

merged = merged.loc[merged.Value <= 1000]
len(merged)

CA-result-over-one-thousand.csv
139


738040

In [35]:
# Same for less than zero

lessThanZero = merged.loc[merged.Value < 0]
lessThanZero.to_csv(makepath(less_than_zero), index=False, quoting = csv.QUOTE_ALL)
print(len(lessThanZero))
del lessThanZero

merged = merged.loc[merged.Value >= 0]
len(merged)

CA-result-less-than-zero.csv
448


737592

In [36]:
merged['ExceedsMclg'] = merged.Value > merged.MCLG

In [37]:
merged.head(10)

,ActivityMediaName,ActivityMediaSubdivisionName,ActivityStartDate,ActivityStartTime/Time,ActivityStartTime/TimeZoneCode,MonitoringLocationIdentifier,CharacteristicName,ResultMeasureValue,ResultMeasure/MeasureUnitCode,ResultCommentText,Value,MeasureGroup,MCLG,Unit,OriginalUnit,Multiplier,OriginalValue,ExceedsMclg
0,Water,NaN,2013-08-06,10:10:00,PST,CALWR_WQX-A1723000,Arsenic,6.08,ug/l,NaN,0.006080,Arsenic,0.000,mg/L,ug/l,0.001,6.080,True
1,Water,NaN,2013-02-26,08:05:00,PST,CALWR_WQX-F0129950,Arsenic,.155,ug/l,NaN,0.000155,Arsenic,0.000,mg/L,ug/l,0.001,0.155,True
2,Water,NaN,2013-08-06,07:30:00,PST,CALWR_WQX-A0263000,Selenium,0,ug/l,NaN,0.000000,Selenium,0.050,mg/L,ug/l,0.001,0.000,False
3,Water,NaN,2013-08-06,07:30:00,PST,CALWR_WQX-A0263000,Arsenic,1.48,ug/l,NaN,0.001480,Arsenic,0.000,mg/L,ug/l,0.001,1.480,True
4,Water,NaN,2013-08-06,07:45:00,PST,CALWR_WQX-A1168000,Selenium,.36,ug/l,NaN,0.000360,Selenium,0.050,mg/L,ug/l,0.001,0.360,False
5,Water,NaN,2013-08-06,07:45:00,PST,CALWR_WQX-A1168000,Lead,0,ug/l,NaN,0.000000,Lead,0.000,mg/L,ug/l,0.001,0.000,False
6,Water,NaN,2013-08-06,07:45:00,PST,CALWR_WQX-A1168000,Cadmium,0,ug/l,NaN,0.000000,Cadmium,0.005,mg/L,ug/l,0.001,0.000,False
7,Water,NaN,2013-05-07,10:15:00,PST,CALWR_WQX-A1723000,Selenium,0,ug/l,NaN,0.000000,Selenium,0.050,mg/L,ug/l,0.001,0.000,False
8,Water,NaN,2013-05-07,10:15:00,PST,CALWR_WQX-A1723000,Copper,.24,ug/l,NaN,0.000240,Copper,1.300,mg/L,ug/l,0.001,0.240,False
9,Water,NaN,2013-05-07,10:15:00,PST,CALWR_WQX-A1723000,Chromium,1.5,ug/l,NaN,0.001500,Chromium,0.100,mg/L,ug/l,0.001,1.500,False


In [38]:
keepers = merged[['ActivityMediaName', 'ActivityMediaSubdivisionName', 'ActivityStartDate', 
                      'ActivityStartTime/Time', 'ActivityStartTime/TimeZoneCode', 
                      'MonitoringLocationIdentifier', 'CharacteristicName', 'MeasureGroup',
                      'Unit', 'Value',  'MCLG', 'ExceedsMclg', 'ResultCommentText']]

keepers.columns = ['Medium', 'MediumSubdivision', 'StartDate', 'StartTime', 'TimeZone',
                  'LocationIdentifier', 'Pollutant', 'PollutantGroup', 'Unit', 'Value', 
                  'Mclg', 'ExceedsMclg', 'Comment']

In [39]:
keepers.to_csv(makepath(result_clean), index=False, quoting = csv.QUOTE_ALL)

CA-result-result-clean.csv
